In [ ]:
!pip install requests
!pip install beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
"""
Function to return true if the book is present in the list being parsed through the goodreads link.
It takes the link, book name, author name in the arguments.
"""
def is_the_book_present(gr_link,book_name,author_name):
  #flag to be returned
  book_flag = False
  gr_page = requests.get(gr_link)
  gr_soup = BeautifulSoup(gr_page.content,"html.parser")
  #class with the list found from page source
  gr_tb_results = gr_soup.find(class_ = "tableList js-dataTooltip")
  try:
    #get all the book entries
    gr_rows = gr_tb_results.find_all("tr", itemtype = "http://schema.org/Book")
    #go through all the book rows and check columns to fetch book name and author name to compare with arguments
    for gr_row in gr_rows:
      gr_entry = gr_row.find("td", width = "100%")
      gr_book_item = gr_entry.find_all("span", itemprop="name")
      gr_book = gr_book_item[0].text.strip().lower()
      gr_author = gr_book_item[1].text.strip().lower()
      #return true if match found
      if book_name in gr_book and author_name == gr_author:
        book_flag = True
        break
    return book_flag
  except AttributeError:
      return book_flag

In [ ]:
#fetch names
book_name = input("Enter book name: ").lower()
author_name = input("Enter author name: ").lower()

In [6]:
url = "https://www.the52book.club/2024-goodreads-lists/"
page = requests.get(url)

In [7]:
soup = BeautifulSoup(page.content,"html.parser")
results = soup.find(class_ = "entry-content clear")

In [8]:
links = results.find_all("a")
category_dict = {}

In [9]:
#populate list links with challenge categories
for link in links[:-4]:
  category_dict[link['href']] = link.text.strip()

In [ ]:
keys = list(category_dict.keys())
#list to get all the categories where the book is present
found_categories = []
for category in keys:
  if is_the_book_present(category,book_name,author_name):
    found_categories.append(category_dict[category])
#print final results
print(book_name.capitalize() + " by " + author_name.capitalize() + "\n"+ str(found_categories))